# Busca de Custo Uniforme


## 📘 1. Introdução

A Busca de Custo Uniforme é um algoritmo de busca não informada baseado na expansão do nó de menor custo acumulado.

## 📌 Características:
- Estratégia: expansão pelo menor custo $g(n)$
- Estrutura de dados: fila de prioridade
- Completa: Sim
- Ótima: Sim (para custos positivos)
- Complexidade de tempo: $O(b^{1 + C^*/ϵ})$
- Complexidade de espaço: $O(b^{1 + C^*/ε})$

## 📘 2. Estrutura Básica do Problema

### Representação dos nós (estados)

In [1]:
class Estado:
    """
    Representa um estado no espaço de busca de um problema.

    A estrutura é genérica, podendo ser utilizada em problemas como o quebra-cabeça de 8 peças,
    labirintos, jogos e outros domínios que envolvem transições de estado.

    A representação do estado é feita por meio de um vetor unidimensional.
    """

    def __init__(self, vetor=None, custo=0, acao=None, pai=None):
        """
        Inicializa um novo estado.

        Args:
            vetor (list[int], opcional): representação linear do estado (ex: 9 elementos do 8-puzzle).
            custo (int): custo acumulado para atingir este estado a partir do estado inicial.
            acao (str): ação que resultou neste estado a partir do estado pai (ex: 'cima', 'direita').
            pai (Estado): referência ao estado anterior (para reconstrução do caminho da solução).
        """
        if vetor is None:
            vetor = []
        self.vetor = vetor
        self.custo = custo
        self.acao = acao
        self.pai = pai

    def __getitem__(self, i):
        """
        Permite o acesso direto aos elementos do vetor via indexação.

        Args:
            i (int): índice do elemento a ser acessado.

        Returns:
            int: valor na posição i do vetor.
        """
        return self.vetor[i]

    def __eq__(self, outro):
        """
        Define igualdade entre dois estados com base em seus vetores.

        Args:
            outro (Estado): estado a ser comparado.

        Returns:
            bool: True se os vetores forem iguais, False caso contrário.
        """
        if not isinstance(outro, Estado):
            return False
        return self.vetor == outro.vetor

    def __hash__(self):
        """
        Permite o uso de instâncias de Estado como chaves de dicionário ou em conjuntos.

        Returns:
            int: valor hash derivado do vetor (convertido em tupla).
        """
        return hash(tuple(self.vetor))

    def __repr__(self):
        """
        Retorna uma representação legível do vetor do estado, útil para debug.

        Returns:
            str: representação textual do vetor do estado.
        """
        return str(self.vetor)

    def __lt__(self, outro):
        """
        Compara este estado com outro estado com base no custo acumulado g(n).

        Esse método é necessário para utilizar objetos da classe Estado
        em filas de prioridade (como o heapq), que requerem ordenação.

        Args:
            outro (Estado): Outro estado a ser comparado com este.

        Returns:
            bool: True se o custo deste estado for menor que o custo do outro.
        """
        return self.custo < outro.custo

### Representação do Problema

In [2]:
class Problema:
    """
    Classe base para a modelagem de problemas de busca no espaço de estados.

    Esta classe define a interface e estrutura mínima que deve ser seguida por qualquer
    problema a ser resolvido com algoritmos de busca, como Busca em Largura, A*, entre outros.
    """

    def __init__(self):
        """
        Inicializa o problema com estado inicial nulo.
        Subclasses devem sobrescrever essa inicialização.
        """
        self._estado_inicial = None

    @property
    def estado_inicial(self):
        """
        Retorna o estado inicial do problema.

        Returns:
            Estado: o estado de partida.
        """
        if self._estado_inicial is None:
            raise NotImplementedError("O estado inicial não foi definido.")
        return self._estado_inicial

    def estado_objetivo(self, estado):
        """
        Verifica se o estado fornecido atende à condição de objetivo.

        Args:
            estado (Estado): estado a ser testado.

        Returns:
            bool: True se é um estado objetivo, False caso contrário.
        """
        raise NotImplementedError("Método 'estado_objetivo' deve ser implementado na subclasse.")

    def solucao(self, estado):
        """
        Reconstrói o caminho do estado inicial até o estado objetivo, percorrendo os pais.

        Args:
            estado (Estado): estado objetivo atingido.

        Returns:
            list[Estado]: sequência de estados do caminho da solução.
        """
        resultado = []
        ptr = estado
        while ptr:
            resultado.append(ptr)
            ptr = ptr.pai
        return list(reversed(resultado))

    def funcao_sucessora(self, estado):
        """
        Retorna a lista de estados sucessores (vizinhos) do estado atual,
        aplicando todas as ações válidas possíveis.

        Args:
            estado (Estado): estado atual.

        Returns:
            list[Estado]: estados sucessores gerados.
        """
        raise NotImplementedError("Método 'funcao_sucessora' deve ser implementado na subclasse.")


## 📘 3. Implementação da Busca de Custo Uniforme

In [3]:
import heapq

def busca_custo_uniforme(problema: Problema):
    """
    Executa o algoritmo de Busca de Custo Uniforme (Uniform Cost Search - UCS)
    para resolver um problema de busca no espaço de estados.

    Esta estratégia expande sempre o estado de menor custo acumulado g(n).

    Args:
        problema (Problema): Instância de um problema com métodos:
            - estado_inicial (Estado)
            - estado_objetivo(estado)
            - funcao_sucessora(estado)

    Returns:
        list[Estado]: Lista de estados que compõem o caminho da solução,
                      do estado inicial ao objetivo.

    Raises:
        RuntimeError: Se a fila de prioridade esvaziar sem encontrar uma solução.
    """

    # 1. Inicializa a fila de prioridade com o estado inicial
    borda = []
    heapq.heappush(borda, problema.estado_inicial)

    # 2. Inicializa o conjunto de estados já visitados
    memoria = set()
    memoria.add(tuple(problema.estado_inicial.vetor))

    # 3. Loop principal da busca
    while borda:

        # 4. Remove o estado de menor custo acumulado (g(n))
        estado = heapq.heappop(borda)

        # 5. Verifica se o estado atual é o objetivo
        if problema.estado_objetivo(estado):
            return problema.solucao(estado)

        # 6. Gera os estados sucessores
        vizinhos = problema.funcao_sucessora(estado)

        # 7. Adiciona vizinhos não visitados à fila de prioridade
        for vizinho in vizinhos:
            chave = tuple(vizinho.vetor)
            if chave not in memoria:
                heapq.heappush(borda, vizinho)
                memoria.add(chave)

    # 8. Nenhuma solução encontrada
    raise RuntimeError("Falha ao encontrar solução.")


## 📘 4. Exemplo - Problema do Robô em terreno

In [4]:
class ProblemaRoboEmTerreno(Problema):
    """
    Problema de um robô que se movimenta em um grid 2D com custos variáveis de entrada.

    O objetivo é encontrar o caminho de menor custo do ponto de partida até o destino.
    """

    def __init__(self, terreno, inicio, destino):
        """
        Inicializa o problema.

        Args:
            terreno (list[list[int]]): Matriz com os custos de entrada em cada célula.
            inicio (tuple[int, int]): Coordenadas (linha, coluna) da posição inicial.
            destino (tuple[int, int]): Coordenadas (linha, coluna) da posição destino.
        """
        self.terreno = terreno
        self.inicio = inicio
        self.destino = destino
        self.linhas = len(terreno)
        self.colunas = len(terreno[0])
        self._estado_inicial = Estado(vetor=inicio, custo=0)

    @property
    def estado_inicial(self):
        """Retorna o estado inicial do problema."""
        return self._estado_inicial

    def estado_objetivo(self, estado):
        """
        Verifica se o estado atual é o destino.

        Args:
            estado (Estado): Estado atual.

        Returns:
            bool: True se é o destino, False caso contrário.
        """
        return estado.vetor == self.destino

    def funcao_sucessora(self, estado):
        """
        Gera os estados vizinhos acessíveis a partir da posição atual.

        Args:
            estado (Estado): Estado atual com coordenadas (linha, coluna).

        Returns:
            list[Estado]: Lista de estados sucessores com custo acumulado atualizado.
        """
        sucessores = []
        i, j = estado.vetor

        movimentos = {
            'cima': (-1, 0),
            'baixo': (1, 0),
            'esquerda': (0, -1),
            'direita': (0, 1)
        }

        for acao, (di, dj) in movimentos.items():
            ni, nj = i + di, j + dj

            if 0 <= ni < self.linhas and 0 <= nj < self.colunas:
                custo_movimento = self.terreno[ni][nj]
                novo_estado = Estado(
                    vetor=(ni, nj),
                    custo=estado.custo + custo_movimento,
                    acao=acao,
                    pai=estado
                )
                sucessores.append(novo_estado)

        return sucessores


In [5]:
# Definição do Problema
terreno = [
    [1, 1, 2, 3, 9, 4, 1],
    [2, 5, 3, 2, 1, 1, 1],
    [3, 9, 1, 9, 2, 8, 1],
    [1, 1, 1, 1, 3, 3, 2],
    [9, 9, 2, 9, 4, 9, 1],
    [2, 1, 1, 2, 1, 1, 2],
    [1, 3, 9, 1, 1, 5, 1]
]

inicio = (0, 0)      # canto superior esquerdo
destino = (6, 6)     # canto inferior direito

# Execução do problema
problema = ProblemaRoboEmTerreno(terreno, inicio, destino)
caminho = busca_custo_uniforme(problema)

# Impressão simples dos estados no caminho
for estado in caminho:
    print(f"{estado.vetor} ← {estado.acao} (custo acumulado: {estado.custo})")

(0, 0) ← None (custo acumulado: 0)
(0, 1) ← direita (custo acumulado: 1)
(0, 2) ← direita (custo acumulado: 3)
(1, 2) ← baixo (custo acumulado: 6)
(2, 2) ← baixo (custo acumulado: 7)
(3, 2) ← baixo (custo acumulado: 8)
(4, 2) ← baixo (custo acumulado: 10)
(5, 2) ← baixo (custo acumulado: 11)
(5, 3) ← direita (custo acumulado: 13)
(5, 4) ← direita (custo acumulado: 14)
(5, 5) ← direita (custo acumulado: 15)
(5, 6) ← direita (custo acumulado: 17)
(6, 6) ← baixo (custo acumulado: 18)


## 📘 5. Exemplo Visual

In [6]:
!pip install graphviz

In [7]:
from graphviz import Digraph
from IPython.display import display
import heapq
import time

def busca_custo_uniforme_visual(problema: Problema, atraso=1.0):
    """
    Executa a Busca de Custo Uniforme com visualização gráfica da árvore de busca.
    """
    borda = []
    heapq.heappush(borda, problema.estado_inicial)
    memoria = set()
    memoria.add(tuple(problema.estado_inicial.vetor))

    grafo = Digraph(format='png')
    grafo.attr('graph', rankdir='TB')
    grafo.attr('node', shape='box', style='filled,rounded', fontname='monospace',
               fontsize='10', width='0.4', height='0.4', margin='0.05')
    grafo.attr('edge', fontname='monospace', fontsize='10')

    id_nos = {}
    contador_id = 0

    def estado_id(estado):
        nonlocal contador_id
        chave = tuple(estado.vetor)
        if chave not in id_nos:
            id_nos[chave] = f"n{contador_id}"
            contador_id += 1
        return id_nos[chave]

    def formatar_legenda(estado):
        return f"{estado.vetor}\n g={estado.custo}"

    while borda:
        estado = heapq.heappop(borda)
        id_estado = estado_id(estado)

        grafo.node(id_estado, formatar_legenda(estado), fillcolor='gold')
        display(grafo)
        time.sleep(atraso)

        if problema.estado_objetivo(estado):
            return problema.solucao(estado)

        for vizinho in problema.funcao_sucessora(estado):
            chave = tuple(vizinho.vetor)
            id_pai = estado_id(estado)
            id_filho = estado_id(vizinho)

            if chave not in memoria:
                heapq.heappush(borda, vizinho)
                memoria.add(chave)

                grafo.node(id_filho, formatar_legenda(vizinho), fillcolor='palegreen')
                grafo.edge(id_pai, id_filho, label=f"{vizinho.acao}, g={vizinho.custo}")

        grafo.node(id_estado, formatar_legenda(estado), fillcolor='dimgray')

    raise RuntimeError("Falha ao encontrar solução.")


In [8]:
# Descomente o código abaixo para visualizar a execução do algoritmo

#problema = ProblemaRoboEmTerreno(terreno, inicio, destino)
#caminho = busca_custo_uniforme_visual(problema, atraso=0.5)